### Part 5: Customizing State 

Background: you can add additoinal fields to the graph state to define complex  behaviro without relying on the message list (aka free form text)

Objective: build a chatbot to search internet to find specific information, and foward that information to a human for review. 

Example: generate a chatbot to research the birthday of an entity, and add birthday and anme to the keys of the state. 


Note: Adding information to the state method makes it easily accessable by other graph nodes (i.e downstream nodes that store or process information) as well as the graphs persistence layer


#### When to Use **COMMAND** vs **CONDITIONAL EDGES**
- use **command** when you need to *both* update the graph state and route to a different node (ex. multi-agent handoffs, where you need to route to a different agent and pass some info to that agent)
- use **conditional edges** to route between nodes conditionally without updating the state 

In [1]:
#Import API Keys       
_root = "/home/zjc1002/Mounts/code/"

# Specifcy Anthropic model to use 
_model_id = "anthropic:claude-3-5-sonnet-20240620"

In [ ]:
import sys 
from typing import Annotated, Dict 

from langchain.chat_models import init_chat_model
from langchain_tavily import TavilySearch
from langchain_anthropic import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.types import Command, interrupt
from langgraph.prebuilt import MessagesState

import langchain_core
from langchain_core.messages import ToolMessage
from langchain_core.tools import InjectedToolCallId, tool

# Custom functions
sys.path.append(_root)
from  admin.api_keys import _api_keys
from admin.sys_ops import _set_env
from colab_projects.tutorials.langgraph.tools import get_weather

# This anthropic and internet tool requires  API key to be set as a environment variable(s)
for api_key in ["TAVILY_API_KEY","ANTHROPIC_API_KEY"]:
    _set_env( api_key
            , value =  _api_keys[api_key])

### Defining State Explicitly vs using langgraphs prebuilt MessagesState

```Python
# Note: Below is the deffinition of langgraphs pre-built MessagesState 
class MessagesState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
```

In [ ]:
#Version 1: explicitly defining the messages attriubute as an annotated list using the add_messages reducer
class State(TypedDict):
    """
    State for the graph , including 2 custom string parameters(name, birthday) to enable passing values across graph nodes.
    """

    messages: Annotated[list, add_messages]
    name: str
    birthday: str


#Version 2: use langgraphs prebuilt MessagesSate to abstract the messages attribute
# We no longer need to define the messages attribute in the State class. Typically, there is more state to track than just messages, so we see people subclass this state and add more fields, like:
class State(MessagesState):
    """
    State for the graph , including 2 custom string parameters(name, birthday) to enable passing values across graph nodes.
    """
    name: str
    birthday: str

In [ ]:
@tool
# Note that because we are generating a ToolMessage for a state update, we
# generally require the ID of the corresponding tool call. We can use
# LangChain's InjectedToolCallId to signal that this argument should not
# be revealed to the model in the tool's schema.

# OBJECTIVE: GENERATE STATE UPDATES FROM INSIDE THE TOOL
def human_assistance(
    name: str
    , birthday: str
    , tool_call_id: Annotated[str, InjectedToolCallId]
) -> str:
    
    """Request assistance from a human."""

    print('AI MODEL THINKS NAME IS:', name)
    print('AI MODEL THINKS BIRTHDAY IS:', birthday)

    # We can use the interrupt function to ask a human reviewer to
    human_response = interrupt(
        {
            "question": "Is this correct?",
            "name": name,
            "birthday": birthday,
        },
    )
    # If the information is correct, update the state as-is.
    if human_response.get("correct", "").lower().startswith("y"):
        print("Human reviewer confirmed the information.")
        verified_name = name
        verified_birthday = birthday
        response = "Correct"
        
    # Otherwise, receive information from the human reviewer.
    else:
        print("Human reviewer provided corrections.")
        verified_name = human_response.get("name", name)
        verified_birthday = human_response.get("birthday", birthday)
        response = f"Made a correction: {human_response}"

    # This time we explicitly update the state with a ToolMessage inside
    # the tool.
    state_update = {
        "name": verified_name,
        "birthday": verified_birthday,
        "messages": [ToolMessage(response, tool_call_id=tool_call_id)],
    }


    # We return a Command object in the tool to update our state.
    return Command(update=state_update)


def chatbot(state: State) -> Command:
    """Process user inputs and generate AI responses in the conversation flow.
    
    This function handles the core chatbot functionality by invoking the LLM with 
    the current conversation context. It ensures proper tool handling by limiting
    to one tool call at a time to support the human-in-the-loop workflow.
    
    Args:
        state (State): The current conversation state containing message history
        llm_with_tools (RunnableBinding): The language model with bound tools
        
    Returns:
        Command: Dictionary containing the AI response message
    """
    # Process the conversation state and generate a response
    message = llm_with_tools.invoke(state["messages"])
    assert(len(message.tool_calls) <= 1)
    return {"messages": [message]}


def stream_graph_updates(user_input:str , graph ,  config:Dict): 
    
    """Stream updates from a LangGraph execution and print the responses.
    
    This function processes a user input through a LangGraph conversation flow,
    streaming the results as they become available and printing each message.
    
    Args:
        user_input (str): The user's question or instruction to process
        graph (CompiledStateGraph): The compiled LangGraph object that defines the conversation flow
        config (Dict): Configuration settings for the graph execution, including thread ID
    """

    events = graph.stream(
        {'messages': [{'role': "user", "content": user_input}]}
         , config
         , stream_mode = 'values'
         )
    
    # Print the messages as they are received
    for event in events:
        if "messages" in event:
            event["messages"][-1].pretty_print()


def stream_graph_updates_human_input(human_response:str
                                     , graph 
                                     ,  config:Dict): 
    """j
    Stream updates from a LangGraph execution and print the responses.
    """
    
    events = graph.stream(
        (human_response)
         , config
         , stream_mode = 'values'
         )
    
    for event in events:
        if "messages" in event:
            event["messages"][-1].pretty_print()
                


In [5]:

#specify tools 
internet_search = TavilySearch(max_results=2)
tools = [internet_search, human_assistance, get_weather]

#compile tools and llm 
llm = init_chat_model(_model_id) #generalized method to load any supported decoder for use as chatbot 
llm_with_tools = llm.bind_tools(tools)
tool_node = ToolNode(tools=tools)

######
#CREATE THE GRAPH
######

#compile the graph
graph_builder = StateGraph(State)

#add chatbot and tools to the graph
graph_builder.add_node("chatbot", chatbot)

#add tool node to the graph
graph_builder.add_node("tools", tool_node)

# TOOLS are processed and evaluated for use via conditional edges
# The tools_condition function is a custom function that determines whether the chatbot should use a tool based on the current state of the conversation.
# When add_conditional_edges is called with the "chatbot" node and this function, it's establishing the logic that determines where the conversation flow should go after the chatbot node processes a message.
# This creates a decision point in the graph where the conversation can either:Route to the tools node if the chatbot determines it needs external information or Loop back to the chatbot or go to another node if no tools are needed
# This conditional routing allows the chatbot to dynamically decide when to use the provided tools (internet search, human assistance, or weather information) based on the conversation context.
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)

graph_builder.add_edge("tools", "chatbot") #generates / creates a path from tools to chatbot
graph_builder.add_edge(START, "chatbot") #defines the starting point of the graph

memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

In [6]:
#inital pass at trying to identify the name and birthday of langgraph
user_input = (
    "Can you look up when LangGraph was released? "
    "When you have the answer, use the human_assistance tool for review"
)

config = {"configurable": {"thread_id": "1"}}


stream_graph_updates(user_input , graph ,  config)

================================ Human Message =================================

Can you look up when LangGraph was released? When you have the answer, use the human_assistance tool for review
================================== Ai Message ==================================

[{'text': "Certainly! I'll search for information about LangGraph's release date and then use the human_assistance tool for review. Let's start with the search.", 'type': 'text'}, {'id': 'toolu_01WYwZyEAwa2C3eAnLjPQyP5', 'input': {'query': 'When was LangGraph released?'}, 'name': 'tavily_search', 'type': 'tool_use'}]
Tool Calls:
  tavily_search (toolu_01WYwZyEAwa2C3eAnLjPQyP5)
 Call ID: toolu_01WYwZyEAwa2C3eAnLjPQyP5
  Args:
    query: When was LangGraph released?
================================= Tool Message =================================
Name: tavily_search

{"query": "When was LangGraph released?", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "LangGraph Quickstart", "url":

In [7]:
def get_structured_human_input():
    """
    Get structured input from a human in dictionary format.
    Returns a Command object with the correct format for resume data.
    """
    print("\nPlease provide your response in the following format:")
    print("correct=yes/no; name=NAME; birthday=DATE")
    print("Example: correct=yes; name=LangGraph; birthday=Jan 17, 2024")
    print("Or just 'correct=yes' if all information is correct")
    
    user_input = input("\nYour response: ")
    
    # Parse the input
    response = {}
    if "correct=yes" in user_input.lower():
        response["correct"] = "yes"
    else:
        response["correct"] = "no"
        
    # Extract name and birthday if provided
    if "name=" in user_input:
        name_part = user_input.split("name=")[1].split(";")[0].strip()
        response["name"] = name_part
    
    if "birthday=" in user_input:
        birthday_part = user_input.split("birthday=")[1].split(";")[0].strip()
        response["birthday"] = birthday_part
    
    return response

# Get structured input from the human
human_input = get_structured_human_input()
print("Structured input received:", human_input)


Please provide your response in the following format:
correct=yes/no; name=NAME; birthday=DATE
Example: correct=yes; name=LangGraph; birthday=Jan 17, 2024
Or just 'correct=yes' if all information is correct
Structured input received: {'correct': 'no', 'name': 'LangGraph', 'birthday': '10/02/2023'}


In [8]:
#the model cannot identify the birthday and name of LangGraph, so we use the structured response from human
human_command = Command(resume=human_input, )
human_command

Command(resume={'correct': 'no', 'name': 'LangGraph', 'birthday': '10/02/2023'})

In [9]:
stream_graph_updates_human_input(human_command
                                     , graph 
                                     ,  config)

================================== Ai Message ==================================

[{'text': "Thank you for providing that information. Based on the search results, I couldn't find a specific release date for LangGraph. The search results don't provide a clear answer to when LangGraph was released. Let's use the human_assistance tool to get more accurate information and review.", 'type': 'text'}, {'id': 'toolu_01QyERRBMXnNeM1gEm6fSA4f', 'input': {'name': 'Assistant', 'birthday': '2023-01-01'}, 'name': 'human_assistance', 'type': 'tool_use'}]
Tool Calls:
  human_assistance (toolu_01QyERRBMXnNeM1gEm6fSA4f)
 Call ID: toolu_01QyERRBMXnNeM1gEm6fSA4f
  Args:
    name: Assistant
    birthday: 2023-01-01
AI MODEL THINKS NAME IS: Assistant
AI MODEL THINKS BIRTHDAY IS: 2023-01-01
Human reviewer provided corrections.
================================= Tool Message =================================
Name: human_assistance

Made a correction: {'correct': 'no', 'name': 'LangGraph', 'birthday': '10/02/2

In [12]:
#we manually set the name and birthday to LangGraph and Jan 17, 2024
#and then pass the command to the graph for processing
events = graph.stream(human_command
                      , config, stream_mode="values")
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================== Ai Message ==================================

Thank you for the human assistance. It appears that the information I found was not entirely accurate. The human has provided a correction:

LangGraph was actually released on January 1, 2022 (2022-01-01).

This date is earlier than what we found in our initial search, which is not unusual for software projects. Sometimes, the first public release or announcement might come after the initial development or internal release.

To summarize:
- LangGraph was released on January 1, 2022.
- The information from our initial search about recent versions and ongoing development is still relevant, showing that the project is actively maintained and updated.

Is there anything else you'd like to know about LangGraph or its release?


In [58]:
#view the updated state of the graph
snapshot = graph.get_state(config)
{k: v for k, v in snapshot.values.items() if k in ("name", "birthday")}

{'name': 'LangGraph', 'birthday': '10/02/1988'}

#### Manually Updating a graphs state  
*A) at any point, we can manually override a key in the state of the graph*

*B) after setting state, if we call get_state(with correct config) we can see the updated values reflected in the graph state*

##### WARNING: Use of the interrupt function is generally recommended instead, as it allows data to be transmitted in a human-in-the-loop interaction independently of state updates.

In [59]:
#A
graph.update_state(config,{'name': 'LangGraph(test)', 'birthday': 'Jan 17, 2024'})

#B
snapshot = graph.get_state(config)
{k: v for k, v in snapshot.values.items() if k in ("name", "birthday")}

{'name': 'LangGraph(test)', 'birthday': 'Jan 17, 2024'}

### Part 6: Time Travel 
- enabling  model to start from any prior response, and *branch off* to explore alternate outcomes
- enable useres to *rewind* the chatbots work to fix mistakes or try a new approach


##### **Objective:** rewind your graph by fetching a checkpoint using the graphs **get_state_history()** method. then resume execution from this previous state of the conversation 

In [10]:
#replay the full state history to see everything that occurred.
#Notice that checkpoints are saved for every step of the graph. This spans invocations so you can rewind across a full thread's history.
    
to_replay = None
for state in graph.get_state_history(config):
    print("Num Messages: ", len(state.values["messages"]), "Next: ", state.next)
    print("-" * 80)
    if len(state.values["messages"]) == 6:
        # We are somewhat arbitrarily selecting a specific state based on the number of chat messages in the state.
        to_replay = state


Num Messages:  6 Next:  ()
--------------------------------------------------------------------------------
Num Messages:  5 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num Messages:  4 Next:  ('tools',)
--------------------------------------------------------------------------------
Num Messages:  3 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num Messages:  2 Next:  ('tools',)
--------------------------------------------------------------------------------
Num Messages:  1 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num Messages:  0 Next:  ('__start__',)
--------------------------------------------------------------------------------


In [11]:
#the checkpoint's config (to_replay.config) contains a checkpoint_id timestamp. 
# Providing this checkpoint_id value tells LangGraph's checkpointer to load the state from that moment in time

print(to_replay.next)
print(to_replay.config)
print(to_replay.values["messages"][-1].content)

()
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f02b659-d881-6d77-8005-f5d27be87c8a'}}
Thank you for the human assistance. I apologize for the confusion in my previous response. The human assistance has provided a correction regarding LangGraph's release date. 

According to the human assistance:

LangGraph was released on October 2, 2023 (10/02/2023).

This information is more specific and accurate than what I initially found through the search. It's important to note that this date represents the initial release of LangGraph.

Is there anything else you would like to know about LangGraph or its release?


In [12]:
# The `checkpoint_id` in the `to_replay.config` corresponds to a state we've persisted to our checkpointer.
for event in graph.stream(None, to_replay.config, stream_mode="values"):
    if "messages" in event:
        event["messages"][-1].pretty_print()





================================== Ai Message ==================================

Thank you for the human assistance. I apologize for the confusion in my previous response. The human assistance has provided a correction regarding LangGraph's release date. 

According to the human assistance:

LangGraph was released on October 2, 2023 (10/02/2023).

This information is more specific and accurate than what I initially found through the search. It's important to note that this date represents the initial release of LangGraph.

Is there anything else you would like to know about LangGraph or its release?


In [16]:
for event in graph.stream(
    
    {'messages':[{'role':'user','content':'YES, please tell me how to tie shoes'}]}
     ,  to_replay.config, stream_mode="values"):
    if "messages" in event:
        event["messages"][-1].pretty_print()

for event in graph.stream(
    
    {'messages':[{'role':'user','content':'YES,  who invented the shoe?'}]}
     ,  to_replay.config, stream_mode="values"):
    if "messages" in event:
        event["messages"][-1].pretty_print()
        

================================ Human Message =================================

YES, please tell me how to tie shoes
================================== Ai Message ==================================

[{'text': "Certainly! I'd be happy to explain how to tie shoes. Since this is a general knowledge question and doesn't require real-time information, I'll provide you with a step-by-step guide. However, to ensure the information is accurate and up-to-date, let's use the Tavily search engine to double-check the steps.", 'type': 'text'}, {'id': 'toolu_01WMStjsw6tmWgH1ZYdxxd6x', 'input': {'query': 'How to tie shoes step by step'}, 'name': 'tavily_search', 'type': 'tool_use'}]
Tool Calls:
  tavily_search (toolu_01WMStjsw6tmWgH1ZYdxxd6x)
 Call ID: toolu_01WMStjsw6tmWgH1ZYdxxd6x
  Args:
    query: How to tie shoes step by step
================================= Tool Message =================================
Name: tavily_search

{"query": "How to tie shoes step by step", "follow_up_questions": nu

In [18]:
for state in graph.get_state_history(config):
    print("Num Messages: ", len(state.values["messages"]), "Next: ", state.next)
    print("-" * 80)
#    if len(state.values["messages"]) == 6:
        # We are somewhat arbitrarily selecting a specific state based on the number of chat messages in the state.
#        to_replay = state

Num Messages:  10 Next:  ()
--------------------------------------------------------------------------------
Num Messages:  9 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num Messages:  8 Next:  ('tools',)
--------------------------------------------------------------------------------
Num Messages:  7 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num Messages:  6 Next:  ('__start__',)
--------------------------------------------------------------------------------
Num Messages:  10 Next:  ()
--------------------------------------------------------------------------------
Num Messages:  9 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num Messages:  8 Next:  ('tools',)
--------------------------------------------------------------------------------
Num Messages:  7 Next:  ('chatbot',)
---------------------------------------